<a href="https://colab.research.google.com/github/terisuke/colab/blob/main/job_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update -qq
!apt-get install -y chromium-chromedriver -qq
!pip install selenium gspread google-auth beautifulsoup4 lxml spacy requests pykakasi backoff pandas -q
!pip install -q google-generativeai
!python -m spacy download ja_core_news_sm

In [ ]:
import time
import re
import gspread
import requests
import spacy  # 日本語の姓名分割に使用
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from google.colab import auth
from google.auth import default
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException
from pykakasi import kakasi
import json
import os
import pandas as pd
import logging
import backoff
import random
from typing import Optional, Tuple, List, Dict, Any, Set
from urllib.parse import urlparse
import google.generativeai as genai

########################################
# ロギング設定（必要に応じてレベルを変更）
########################################
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

########################################
# 環境変数などの設定
########################################
os.environ["HUNTER_API_KEY"] = "YOUR_HUNTER_API_KEY"   # 実際のキーを入れてください
PERPLEXITY_API_KEY = "YOUR_PERPLEXITY_API_KEY"         # 実際のキーを入れてください
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY"                 # 実際のキーを入れてください

SPREADSHEET_NAME = "Hakata_Job_Data"   # 既存または新規作成するスプレッドシート名
JOB_SHEET_NAME = "求人情報"            # 求人を保存するシート名
OFFICER_SHEET_NAME = "役員情報"        # 役員情報を保存するシート名
MAX_LINKS = 10    # スクレイピングする求人情報の最大数
SAVE_INTERVAL = 5 # 何件ごとに進捗を保存するか

replacement_map = {
    '髙橋': '高橋',
    '渡邉': '渡辺',
    # 必要に応じて追加
}

########################################
# Googleスプレッドシート認証
########################################
def authenticate_google_sheets():
    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        return gc
    except Exception as e:
        logging.error(f"Google Sheets認証エラー: {e}")
        raise

########################################
# スプレッドシートとシートの準備
########################################
def get_or_create_sheet(gc, spreadsheet_name, sheet_name):
    try:
        spreadsheet = gc.open(spreadsheet_name)
        logging.info(f"スプレッドシート '{spreadsheet_name}' は既に存在します。")
    except gspread.exceptions.SpreadsheetNotFound:
        spreadsheet = gc.create(spreadsheet_name)
        logging.info(f"スプレッドシート '{spreadsheet_name}' を新規作成しました。")

    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        logging.info(f"シート '{sheet_name}' は既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
        logging.info(f"シート '{sheet_name}' を新規作成しました。")
    return spreadsheet, worksheet

########################################
# Chrome (headless) の起動
########################################
def setup_chrome_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    try:
        driver = webdriver.Chrome(options=options)
        return driver
    except WebDriverException as e:
        logging.error(f"Chrome ドライバの起動に失敗しました: {e}")
        raise SystemExit("処理を続行できません。")

########################################
# ハローワークからの求人情報スクレイピング
########################################
def scrape_job_data(driver, max_links):
    url = "https://www.hellowork.mhlw.go.jp/"
    try:
        driver.get(url)
        time.sleep(2)
    except WebDriverException as e:
        logging.error(f"【エラー】ハローワークサイトにアクセスできませんでした: {e}")
        return []

    try:
        driver.find_element(By.CLASS_NAME, "retrieval_icn").click()  # 「求人情報検索」ボタン
        time.sleep(5)
    except NoSuchElementException as e:
        logging.error(f"【エラー】'求人情報検索' アイコンが見つかりませんでした: {e}")
        return []

    # 福岡県、博多区を選択
    try:
        Select(driver.find_element(By.ID, "ID_tDFK1CmbBox")).select_by_value("40")  # 福岡県
        time.sleep(2)
    except NoSuchElementException as e:
        logging.error(f"【エラー】都道府県選択ができませんでした: {e}")
        return []

    try:
        driver.find_elements(By.CSS_SELECTOR, "input.button")[1].click()  # 「市区町村などでさらに絞り込む」
        time.sleep(2)
    except (NoSuchElementException, IndexError) as e:
        logging.error(f"【エラー】'市区町村などでさらに絞り込む' ボタンが見つかりませんでした: {e}")
        return []

    # 博多区 (40132)
    try:
        Select(driver.find_element(By.ID, "ID_rank1CodeMulti")).select_by_value("40132")
        time.sleep(2)
        driver.find_element(By.ID, "ID_ok").click()
        time.sleep(5)
        driver.find_element(By.ID, "ID_searchBtn").click()
        time.sleep(5)
    except NoSuchElementException as e:
        logging.error(f"【エラー】市区町村選択ができませんでした: {e}")
        return []

    # 1ページあたり50件
    try:
        Select(driver.find_element(By.ID, "ID_fwListNaviDispBtm")).select_by_value("50")
        time.sleep(5)
    except NoSuchElementException:
        logging.warning("1ページあたり件数を選択できませんでした。標準表示件数で続行。")

    all_results = []
    processed_domains = set()

    while True:
        detail_urls, next_page_exists = parse_page(driver, max_links)
        for detail_url in detail_urls:
            if len(all_results) >= max_links:
                break
            detail_info = parse_detail_info(driver, detail_url)
            if detail_info:
                # 同じドメイン重複を避けたい場合にチェック
                if detail_info["email_domain"] not in processed_domains:
                    all_results.append(detail_info)
                    processed_domains.add(detail_info["email_domain"])

        if len(all_results) >= max_links or not next_page_exists:
            break

        try:
            next_btn = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.NAME, "fwListNaviBtnNext"))
            )
            next_btn.click()
            time.sleep(5)
        except TimeoutException:
            logging.warning("次のページボタンが見つからず終了。")
            break

    return all_results

def parse_page(driver, max_links):
    soup = BeautifulSoup(driver.page_source, "html.parser")
    detail_urls = []
    for job in soup.find_all("table", class_="kyujin"):
        detail_btn = job.find("a", string=lambda t: t and "詳細を表示" in t)
        if detail_btn:
            href = detail_btn['href']
            if href.startswith("./"):
                href = href[2:]
            detail_url = "https://www.hellowork.mhlw.go.jp/kensaku/" + href
            detail_urls.append(detail_url)
            if len(detail_urls) >= max_links:
                break

    # 次ページの有無
    try:
        driver.find_element(By.NAME, "fwListNaviBtnNext")
        next_page_exists = True
    except NoSuchElementException:
        next_page_exists = False

    return detail_urls, next_page_exists

def parse_detail_info(driver, url):
    main_window = driver.current_window_handle
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[-1])
    try:
        driver.get(url)
        time.sleep(2)
    except WebDriverException as e:
        logging.error(f"【エラー】求人詳細ページにアクセスできませんでした: {url}, エラー: {e}")
        driver.close()
        driver.switch_to.window(main_window)
        return None

    soup = BeautifulSoup(driver.page_source, "html.parser")

    def get_text_safe(soup_obj, element_id, field_name):
        try:
            return soup_obj.find(id=element_id).get_text(strip=True)
        except AttributeError:
            logging.warning(f"詳細ページで {field_name} が見つかりませんでした: {url}")
            return ""

    company_name = get_text_safe(soup, "ID_jgshMei", "会社名")
    if not company_name:
        logging.warning(f"会社名が取得できなかったためスキップ: {url}")
        driver.close()
        driver.switch_to.window(main_window)
        return None

    exclude_keywords = ["支店", "営業所", "支社", "出張所"]
    if any(keyword in company_name for keyword in exclude_keywords):
        logging.info(f"会社名に除外キーワードが含まれるためスキップ: {company_name}")
        driver.close()
        driver.switch_to.window(main_window)
        return None

    representative_name = get_text_safe(soup, "ID_dhshaMei", "代表者名")
    industry = get_text_safe(soup, "ID_sngBrui", "産業分類")
    email = get_text_safe(soup, "ID_ttsEmail", "メールアドレス")

    if not email:
        logging.warning(f"メールアドレスが取得できなかったためスキップ: {url}")
        driver.close()
        driver.switch_to.window(main_window)
        return None

    if '@' in email:
        email_local, email_domain = email.split('@', 1)
    else:
        email_local = ""
        email_domain = email

    driver.close()
    driver.switch_to.window(main_window)

    return {
        "company_name": company_name,
        "representative_name": representative_name,
        "industry": industry,
        "email_local": email_local,
        "email_domain": email_domain,
        "url": url
    }

########################################
# リトライ機能付きURLフェッチ
########################################
@backoff.on_exception(
    backoff.expo,
    (requests.exceptions.RequestException, Exception),
    max_tries=5,
    max_time=300
)
def fetch_url_with_retry(url):
    time.sleep(random.uniform(1, 3))
    response = requests.get(url)
    if response.status_code == 429:
        raise requests.exceptions.RequestException("Rate limit (429)")
    return response.status_code, response.content

########################################
# ローマ字変換(ヘボン式)の補助
########################################
def convert_kunrei_to_hepburn(text):
    if pd.isna(text) or not isinstance(text, str):
        return text

    conversion_rules = {
        'ou': 'o',
        'oo': 'o',
        'uu': 'u',
        'aa': 'a',
        'si': 'shi',
        'ti': 'chi',
        'tu': 'tsu',
        'zi': 'ji',
        'sya': 'sha', 'syu': 'shu', 'syo': 'sho',
        'tya': 'cha', 'tyu': 'chu', 'tyo': 'cho',
        'dya': 'ja',  'dyu': 'ju',  'dyo': 'jo',
        'n\'a': 'nya',
    }

    try:
        result = text.lower()
        result = re.sub(r'[^a-z0-9]', '', result)  # アルファベット・数字以外除去
        for kunrei, hepburn in conversion_rules.items():
            result = result.replace(kunrei, hepburn)
            result = re.sub(r"n'(?=[aeiouy])", "n", result)  # "n' + 母音" → "n + 母音"
        result = re.sub(r'[^a-z0-9\'-]', '', result)
        return result
    except Exception as e:
        logging.error(f"変換エラー（{text}）: {str(e)}")
        return text

########################################
# 複数の名前候補を取得する (カラム名を修正)
########################################
def process_name_candidates(df, column_prefix, name_column,
                            processed_indices, output_file, save_interval=10):
    """
    column_prefix が "役員名（姓" あるいは "役員名（名" を想定。
    officer_header の「役員名（姓_候補1）」などと同一にするため、
    カラム名をしっかり合わせる。
    """
    max_candidates = 3

    # column_prefix に「姓」が含まれているかで分岐
    if "姓" in column_prefix:
        # => 役員名（姓_候補{i}）
        prefix_columns = [f"役員名（姓_候補{i}）" for i in range(1, max_candidates + 1)]
    else:
        # => 役員名（名_候補{i}）
        prefix_columns = [f"役員名（名_候補{i}）" for i in range(1, max_candidates + 1)]

    # カラムが存在しなければ作る
    for col in prefix_columns:
        if col not in df.columns:
            df[col] = ""

    url = "https://kanji.reader.bz/{name}"
    records_processed = 0

    for index, row in df.iterrows():
        if index in processed_indices:
            continue

        name = row[name_column]
        # 空ならスキップ
        if not isinstance(name, str):
            name = str(name)
        if not name.strip():
            continue

        logging.info(f"Index {index}: 名前候補取得開始 => {name}")

        try:
            status_code, content = fetch_url_with_retry(url.format(name=name))
            if status_code == 200:
                soup = BeautifulSoup(content, 'html.parser')
                yomikata_element = soup.find('p', id='yomikata')

                if yomikata_element:
                    text_content = yomikata_element.get_text(separator=' ', strip=True)
                    words = text_content.split()

                    romaji_candidates = []
                    current_candidate = ""

                    for word in words:
                        # ひらがな含まない => ローマ字とみなす
                        if not re.search('[\u3040-\u309F]', word):
                            if current_candidate:
                                romaji_candidates.append(current_candidate)
                                current_candidate = ""
                            current_candidate = word
                        else:
                            if current_candidate:
                                romaji_candidates.append(current_candidate)
                                current_candidate = ""

                    if current_candidate:
                        romaji_candidates.append(current_candidate)

                    # 姓は先頭から, 名は末尾から
                    if "姓" in column_prefix:
                        relevant_candidates = romaji_candidates[:max_candidates]
                    else:
                        relevant_candidates = romaji_candidates[-max_candidates:] if romaji_candidates else []

                    for i, romaji in enumerate(relevant_candidates):
                        if i < max_candidates:
                            hepburn_romaji = convert_kunrei_to_hepburn(romaji)
                            df.at[index, prefix_columns[i]] = hepburn_romaji
            else:
                print(f"Kanji.reader.bz フェッチエラー: {status_code}")
        except Exception as e:
            logging.error(f"Error processing name {name}: {str(e)}")

        processed_indices.add(index)
        records_processed += 1

        if records_processed % save_interval == 0:
            save_progress(df, output_file)
            print(f"中間保存完了: {records_processed}件処理済み")

    save_progress(df, output_file)
    return df

########################################
# 進捗管理と永続化
########################################
def load_progress(output_file):
    processed_sei_indices = set()
    processed_mei_indices = set()
    if os.path.exists(output_file):
        try:
            df = pd.read_csv(output_file)
            if '役員名（姓_候補1）' in df.columns:
                processed_sei_indices = set(df[df['役員名（姓_候補1）'].notna()].index)
            if '役員名（名_候補1）' in df.columns:
                processed_mei_indices = set(df[df['役員名（名_候補1）'].notna()].index)
            logging.info(f"既存の進捗読込 => 姓{len(processed_sei_indices)}件, 名{len(processed_mei_indices)}件")
            return df, processed_sei_indices, processed_mei_indices
        except Exception as e:
            logging.error(f"進捗ファイル読み込み失敗: {e}")
            return pd.DataFrame(), set(), set()
    return pd.DataFrame(), set(), set()

def save_progress(df, output_file):
    try:
        df.to_csv(output_file, encoding='utf-8-sig', index=False)
        logging.info(f"進捗を保存しました: {output_file}")
    except Exception as e:
        logging.error(f"進捗の保存に失敗: {e}")

########################################
# Perplexity API関連
########################################
def get_officers_from_perplexity(company_name):
    payload = {
        "model": "sonar-pro",
        "messages": [
            {
                "role": "user",
                "content": (
                    f"Search for information about the executive officers of {company_name}. "
                    "Provide any relevant details, including names, titles, and possibly other related information."
                )
            }
        ],
        "max_tokens": 2048,
        "temperature": 0.2
    }
    headers = {
        "Authorization": f"Bearer {PERPLEXITY_API_KEY}",
        "Content-Type": "application/json",
        "Accept": "application/json"
    }
    try:
        resp = requests.post("https://api.perplexity.ai/chat/completions", headers=headers, json=payload)
        resp.raise_for_status()
        result_json = resp.json()
        if "choices" in result_json and len(result_json["choices"]) > 0:
            content = result_json["choices"][0]["message"]["content"]
            logging.info(f"[Perplexity] Raw text => {content[:200]}...")  # 先頭200文字だけ出力
            return content if content else ""
        else:
            logging.warning("Perplexity応答に'choices'なし or 空")
            return ""
    except Exception as e:
        logging.error(f"Perplexity API呼び出しエラー: {e}")
        return ""

########################################
# Gemini API呼び出しを指数バックオフ
########################################
def call_gemini_api_with_backoff(model, prompt, generation_config, max_retries=5):
    backoff_time = 1
    for attempt in range(1, max_retries + 1):
        try:
            response = model.generate_content(prompt, generation_config=generation_config)
            return response
        except genai.errors.RequestError as e:
            if hasattr(e, 'message') and 'quota' in e.message.lower():
                logging.warning(f"Gemini API 429 / Quotaエラー: {attempt}回目。{backoff_time}秒待機。")
                time.sleep(backoff_time)
                backoff_time *= 2
            else:
                logging.error(f"Gemini APIリクエストエラー: {e}")
                return None
        except Exception as ex:
            logging.error(f"Gemini API呼び出し中エラー: {ex}")
            return None
    return None

########################################
# Gemini APIを使って役員情報を抽出
########################################
def get_officers_from_gemini(raw_text, company_name=None):
    if not raw_text.strip():
        logging.info("[Gemini] raw_textが空のため解析スキップ")
        return []
    import re
    import json

    # 箇条書き等を抜き出す
    officer_list_pattern = r'(\n\s*[-•⋅▪\-*]|\n\s*\d+\.|\n\s*[（\(]\d+[）\)])\s*(.+)'
    matches = re.findall(officer_list_pattern, raw_text)
    officer_text = "\n".join([m[1] for m in matches]) if matches else raw_text

    try:
        genai.configure(api_key=GEMINI_API_KEY)
        model = genai.GenerativeModel("gemini-2.0-pro-exp-02-05")
    except Exception as e:
        logging.error(f"Gemini API初期化エラー: {e}")
        return []

    prompt = (
        "You are a highly skilled information extraction assistant specializing in Japanese company officers. "
        "Your ONLY task is to extract officer information from the provided Japanese text and output it as a SINGLE, VALID JSON object. "
        "Follow the structure below precisely. Extract ONLY the following, and ABSOLUTELY NOTHING ELSE:\n"
        "{\n"
        f'  "company_name": "{company_name}",\n'
        '  "officers": [\n'
        "     {\n"
        '      "title": "<Officer Title in Japanese>",\n'
        '      "last_name": "<Officer\'s Last Name in Japanese>",\n'
        '      "first_name": "<Officer\'s First Name in Japanese>",\n'
        '      "last_name_en": "<Officer\'s Last Name in Romaji>",\n'
        '      "first_name_en": "<Officer\'s First Name in Romaji>"\n'
        "     },\n"
        "    ...\n"
        "  ]\n"
        "}\n"
        "  *  **title:** The officer's title in *Japanese*. If the title is in English, IGNORE it.\n"
        "  *  **last_name:** The officer's last name in *Japanese*.\n"
        "  *  **first_name:** The officer's first name in *Japanese*.\n"
        "  *  **last_name_en:** Romaji transliteration of last_name.\n"
        "  *  **first_name_en:** Romaji transliteration of first_name.\n"
        "Output MUST be valid JSON. DO NOT add any other text.\n"
        f"TEXT: {officer_text}"
    )

    generation_config = {
        "temperature": 0.2,
        "max_output_tokens": 2048,
    }

    response = call_gemini_api_with_backoff(model, prompt, generation_config, max_retries=5)
    if not response:
        return []

    gemini_text = response.text
    logging.info(f"[Gemini] Raw response => {gemini_text[:200]}...")
    try:
        match = re.search(r"```json\n(.*)\n```", gemini_text, re.DOTALL)
        if match:
            json_text = match.group(1)
        else:
            json_text = gemini_text

        try:
            officers_data = json.loads(json_text)
        except json.JSONDecodeError as e:
            logging.error(f"Gemini応答JSONパース失敗: {e}")
            logging.error(f"Gemini応答: {json_text}")
            return []

        if (isinstance(officers_data, dict) and
            'officers' in officers_data and
            isinstance(officers_data['officers'], list)):
            # タイトルが正しいかチェック
            filtered_officers = []
            for off in officers_data['officers']:
                title = off.get('title', '')
                if not title or not is_valid_title(title):
                    logging.debug(f"Officer discarded due to invalid title: {title}")
                    continue
                filtered_officers.append(off)
            return filtered_officers
        else:
            logging.warning("Gemini応答が想定フォーマットではありません。")
            return []
    except Exception as e:
        logging.error(f"Gemini応答解析中エラー: {e}")
        return []

def is_valid_title(title):
    invalid_keywords = [
        "Based on", "information", "Company", "Unfortunately", "established", "capital",
        "Representative Director", "Director", "Auditor", "President", "CEO", "CFO", "COO",
        "Executive Officer", "Standing Statutory Auditor", "company's",
        "representative", "Here are", "key details", "following information",
        "provide the following", "search results", "appears to be",
        "also serves as", "main business", "headquarters is located",
        "worth noting", "seems to be", "in charge of", "also serves as",
        "current position", "holds", "shares", "ownership", "does not hold", "previous president",
        "also serves", "main business", "headquarters is located", "appears to have undergone",
        "company name changed", "company focuses on", "search results don't provide",
        "company appears to", "limited and potentially outdated", "suggests that",
        "provides an overview", "executive structure of", "worth noting that",
        "effective", "company specializes in", "specializes in", "appointed as"
    ]
    if not title:
        return False
    # NGワードチェック
    if any(keyword.lower() in title.lower() for keyword in invalid_keywords):
        logging.debug(f"Title discarded by filter: {title}")
        return False
    # 英語のみ/数字や括弧含むならNG
    if re.match(r'^[A-Za-z\s]+$', title):
        logging.debug(f"Title is purely English => discard: {title}")
        return False
    if re.search(r'[\d()]', title):
        logging.debug(f"Title has digits or parentheses => discard: {title}")
        return False
    return True

########################################
# フォールバック: 簡易役員情報パース
########################################
def parse_officer_text(answer_text):
    nlp = spacy.load("ja_core_news_sm")
    results = []
    officer_titles = [
        "代表取締役社長", "代表取締役", "取締役", "監査役", "会長", "社長", "副社長",
        "専務取締役", "常務取締役", "執行役員", "取締役会長", "常勤監査役",
        "社外取締役", "社外監査役", "執行役", "理事長", "理事", "監事",
        "代表理事", "副理事長", "専務理事", "常務理事", "部門長", "本部長",
        "事業部長", "支店長", "所長", "部長", "課長", "室長", "局長"
    ]

    officer_pattern = r'(.+?)\s*[:：、,]\s*(.+)'
    officer_pattern2 = r'(.+?)[（\(](.+?)[）\)]'

    for line in answer_text.split("\n"):
        line = line.strip()
        if not line:
            continue

        role = ""
        full_name = ""

        match = re.match(officer_pattern, line)
        if match:
            role = match.group(1).strip()
            full_name = match.group(2).strip()

        if not match:
            match2 = re.match(officer_pattern2, line)
            if match2:
                full_name = match2.group(1).strip()
                role = match2.group(2).strip()

        if role:
            found = False
            for title in officer_titles:
                if title in role:
                    role = title
                    found = True
                    break
            if not found:
                logging.debug(f"Line does not contain known officer title => {line}")
                continue

        if not match and not match2:
            continue

        doc = nlp(full_name)
        if len(doc) >= 2:
            if doc[-1].text in ("氏"):
                last_name = "".join([token.text for token in doc[:-1]])
                first_name = ""
            else:
                last_name = doc[0].text
                first_name = "".join([token.text for token in doc[1:]])
        else:
            last_name = full_name
            first_name = ""

        if role and is_valid_title(role):
            results.append({
                "title": role,
                "last_name": last_name,
                "first_name": first_name
            })

    return results

########################################
# Hunter.io でメールアドレス検索 (指数バックオフ)
########################################
@backoff.on_exception(
    backoff.expo,
    (requests.exceptions.RequestException,),
    max_tries=5,
    max_time=300
)
def get_hunter_email(first_name_en, last_name_en, domain):
    HUNTER_API_KEY = os.getenv("HUNTER_API_KEY", "")
    if not HUNTER_API_KEY:
        print("[Hunter.io] APIキーが設定されていません。")
        return None

    base_url = "https://api.hunter.io/v2/email-finder"
    params = {
        "domain": domain,
        "first_name": first_name_en,
        "last_name": last_name_en,
        "api_key": HUNTER_API_KEY
    }

    resp = requests.get(base_url, params=params)
    if resp.status_code == 429:
        raise requests.exceptions.RequestException("Hunter.io: 429 Rate limit reached.")
    elif resp.status_code == 200:
        data = resp.json().get("data", {})
        email = data.get("email")
        if email:
            print(f"[Hunter.io] {first_name_en} {last_name_en} @ {domain} => {email}")
            return email
        else:
            print(f"[Hunter.io] 見つかりません: {first_name_en} {last_name_en} @ {domain}")
            return None
    else:
        print(f"[Hunter.io] HTTPエラー: {resp.status_code}")
        return None

########################################
# メイン処理
########################################
def main():
    gc = authenticate_google_sheets()
    spreadsheet, worksheet = get_or_create_sheet(gc, SPREADSHEET_NAME, JOB_SHEET_NAME)
    _, officer_worksheet = get_or_create_sheet(gc, SPREADSHEET_NAME, OFFICER_SHEET_NAME)
    driver = setup_chrome_driver()

    # ハローワークからの求人情報
    try:
        logging.info("ハローワークから求人情報を取得中...")
        job_data = scrape_job_data(driver, MAX_LINKS)
        driver.quit()
    except Exception as e:
        logging.error(f"【エラー】スクレイピング中に例外発生: {e}")
        driver.quit()
        return

    # 会社ごとの辞書
    job_dict = {}
    for job in job_data:
        c = job["company_name"]
        d = job["email_domain"]
        if c not in job_dict:
            job_dict[c] = {
                "representative_name": job["representative_name"],
                "industry_list": [],
                "email_domain_dict": {},
                "url_list": []
            }
        job_dict[c]["industry_list"].append(job["industry"])
        if d not in job_dict[c]["email_domain_dict"]:
            job_dict[c]["email_domain_dict"][d] = []
        job_dict[c]["email_domain_dict"][d].append(job["email_local"])
        job_dict[c]["url_list"].append(job["url"])

    # 求人情報をスプレッドシートに書き込み
    logging.info("求人情報をスプレッドシートに書き込み...")
    header = ["会社名", "代表者名", "産業分類（業種）",
              "メールアドレス(ローカル)", "メールアドレス(ドメイン)", "求人詳細URL"]
    worksheet.update([header], "A1")

    row_data = []
    for company_name, vals in job_dict.items():
        industry_str = ",".join(vals["industry_list"])
        url_str = ",".join(vals["url_list"])
        for domain, local_parts in vals["email_domain_dict"].items():
            email_local_str = ",".join(local_parts)
            row_data.append([
                company_name,
                vals["representative_name"],
                industry_str,
                email_local_str,
                domain,
                url_str
            ])
    if row_data:
        worksheet.update(row_data, "A2")
    logging.info(f"{len(job_dict)} 社の求人情報を転記完了。")

    # 役員情報シートのヘッダー
    officer_header = [
        "会社名",
        "Perplexity取得情報",
        "役職名",
        "役員名（姓）",
        "役員名（名）",
        "役員名（姓_候補1）",
        "役員名（姓_候補2）",
        "役員名（姓_候補3）",
        "役員名（名_候補1）",
        "役員名（名_候補2）",
        "役員名（名_候補3）",
        "Hunter推定メールアドレス1",
        "Hunter推定メールアドレス2",
        "Hunter推定メールアドレス3",
    ]
    officer_worksheet.update([officer_header], "A1")

    nlp = spacy.load("ja_core_news_sm")
    kks = kakasi()

    unique_company_names = list(job_dict.keys())
    output_file = "officer_progress.csv"

    # 既存の進捗CSVを読み込み
    df, processed_sei_indices, processed_mei_indices = load_progress(output_file)
    if df.empty:
        df = pd.DataFrame(columns=officer_header)

    # 欠損を埋めて文字列に
    df = df.fillna("")
    for col in df.columns:
        df[col] = df[col].astype(str)

    # Perplexity->Gemini->フォールバック
    for idx, company_name in enumerate(unique_company_names, start=1):
        logging.info(f"[{idx}/{len(unique_company_names)}] 役員検索: {company_name}")
        raw_text = get_officers_from_perplexity(company_name)
        if not raw_text:
            logging.warning(f" => Perplexity応答なし: {company_name}")
            continue

        gemini_officers = get_officers_from_gemini(raw_text, company_name)
        if gemini_officers:
            for off in gemini_officers:
                title = off.get("title", "")
                lname_jp = off.get("last_name", "")
                fname_jp = off.get("first_name", "")
                row = {
                    "会社名": company_name,
                    "Perplexity取得情報": raw_text,
                    "役職名": title,
                    "役員名（姓）": lname_jp,
                    "役員名（名）": fname_jp,
                    "役員名（姓_候補1）": "",
                    "役員名（姓_候補2）": "",
                    "役員名（姓_候補3）": "",
                    "役員名（名_候補1）": "",
                    "役員名（名_候補2）": "",
                    "役員名（名_候補3）": "",
                    "Hunter推定メールアドレス1": "",
                    "Hunter推定メールアドレス2": "",
                    "Hunter推定メールアドレス3": "",
                }
                df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
        else:
            # フォールバック
            fallback_officers = parse_officer_text(raw_text)
            if fallback_officers:
                for off in fallback_officers:
                    title = off.get("title", "")
                    lname_jp = off.get("last_name", "")
                    fname_jp = off.get("first_name", "")
                    row = {
                        "会社名": company_name,
                        "Perplexity取得情報": raw_text,
                        "役職名": title,
                        "役員名（姓）": lname_jp,
                        "役員名（名）": fname_jp,
                        "役員名（姓_候補1）": "",
                        "役員名（姓_候補2）": "",
                        "役員名（姓_候補3）": "",
                        "役員名（名_候補1）": "",
                        "役員名（名_候補2）": "",
                        "役員名（名_候補3）": "",
                        "Hunter推定メールアドレス1": "",
                        "Hunter推定メールアドレス2": "",
                        "Hunter推定メールアドレス3": "",
                    }
                    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
            else:
                logging.info(f"会社名: {company_name}, fallback_officers も空でした。")

        # 進捗を定期保存
        if idx % SAVE_INTERVAL == 0:
            df = df.fillna("")
            for col in df.columns:
                df[col] = df[col].astype(str)
            save_progress(df, output_file)
            print(f"[中間保存] {idx}社 処理完了")

        time.sleep(1)

    # 姓の候補取得 (process_name_candidates) => カラム名整合済み
    print("姓の候補取得開始...")
    df = process_name_candidates(df, "役員名（姓", "役員名（姓）",
                                 processed_sei_indices, output_file, SAVE_INTERVAL)
    df = df.fillna("")
    for col in df.columns:
        df[col] = df[col].astype(str)

    # 名の候補取得
    print("名の候補取得開始...")
    df = process_name_candidates(df, "役員名（名", "役員名（名）",
                                 processed_mei_indices, output_file, SAVE_INTERVAL)
    df = df.fillna("")
    for col in df.columns:
        df[col] = df[col].astype(str)

    print("Hunter.io でメールアドレス候補を検索します...")
    for index, row in df.iterrows():
        company_name = row["会社名"]
        if company_name not in job_dict:
            continue

        domain_dict = job_dict[company_name]["email_domain_dict"]
        if not domain_dict:
            continue

        # 先頭のドメインを1つだけ試す (例)
        domain = next(iter(domain_dict.keys()))
        if not domain.strip():
            continue

        # ローマ字候補列から
        for i in range(1, 4):
            last_name_romaji = row.get(f"役員名（姓_候補{i}）", "")
            if not isinstance(last_name_romaji, str):
                last_name_romaji = str(last_name_romaji)
            if not last_name_romaji.strip():
                continue

            for j in range(1, 4):
                first_name_romaji = row.get(f"役員名（名_候補{j}）", "")
                if not isinstance(first_name_romaji, str):
                    first_name_romaji = str(first_name_romaji)
                if not first_name_romaji.strip():
                    continue

                print(f" [Hunter.io検索] {last_name_romaji} + {first_name_romaji} @ {domain}")
                guessed_email = get_hunter_email(first_name_romaji, last_name_romaji, domain)

                if guessed_email:
                    col_name = f"Hunter推定メールアドレス{i}"
                    if not df.at[index, col_name]:
                        df.at[index, col_name] = guessed_email
                    # breakしない => 全候補を試す

    save_progress(df, output_file)

    # スプレッドシートに書き込み
    officer_rows = df.values.tolist()
    print(f"Final Officer Data Size: {len(officer_rows)}")
    if officer_rows:
        officer_worksheet.update(officer_rows, "A2")
        logging.info(f"役員情報 {len(officer_rows)} 件分をスプレッドシートに書き込みました。")
    else:
        logging.warning("役員情報がありませんでした。")

    logging.info("処理完了。")


if __name__ == "__main__":
    main()